In [1]:
import pandas as pd
import ipywidgets as widgets
import json
import ipyleaflet
from ipyleaflet import basemaps
from branca.colormap import linear

In [2]:
#Reading csv from data created by parsing orginal JSON file
#CSV contains a count of every kind of crime for each participating city/municipality
crime_df = pd.read_csv('MA_crime_by_agency_mod.csv',index_col=0)

In [3]:
#Load static cities GeoJSON
with open("static_cities.json") as f:
    static_geojson = json.load(f)

In [4]:
#Load dynamic cities GeoJSON
with open("dyn_cities.json") as f:
    dyn_geojson = json.load(f)

In [5]:
# #Create a map with at the specified center, with the ability to use the scroll wheel to zoom
# #Note the center param of Map() uses standard lat/long order of coordinates while the Choropleth() function requires the lat/long order to be reversed
center = (42.16340342422403, -70.60638427734376)
m = ipyleaflet.Map(center = center, scroll_wheel_zoom=True, zoom = 8, max_zoom=11)

In [6]:
#Build to choropleth dictionary that puts together the name of the city with the number of this crimes, in this case, the total
choro_data = dict(zip(crime_df.index.tolist(), crime_df['Total'].tolist()))

In [7]:
#Widget that comes on screen when data is loading
loading = widgets.HTML(value="<b>LOADING DATA</b>")
loading_control = ipyleaflet.WidgetControl(widget=loading, position='bottomleft')

In [8]:
#Defining the function that will be called when the user wants to see a different crime type displayed
#This function builds a new choro dictionary based on the new crime type, changed the choro mins and maxs, and changes the value of the choro_data with the new dict
def choro_layer(change):
    m.add_control(loading_control)
    choro_data = dict(zip(crime_df.index.tolist(), crime_df.fillna(0)[change['new']].tolist()))
    dyn_cities.value_max = max(crime_df.fillna(0)[change['new']].tolist())
    dyn_cities.value_min = min(crime_df.fillna(0)[change['new']].tolist())
    dyn_cities.choro_data = choro_data
    m.remove_control(loading_control)

In [9]:
#Creating the choropleth object of cities that provided data that will be added to the map
dyn_cities = ipyleaflet.Choropleth(
    geo_data=dyn_geojson,
    choro_data=choro_data,
    key_on= 'TOWN',
    colormap=linear.YlOrRd_09,
    border_color='black',
    style={'fillOpacity':0.8, 'dashArray': '5,5'})

In [10]:
#Add Chorpleth layer to the map
m.add_layer(dyn_cities)

In [11]:
#Creating the GeoJSON layer of the static cities that did not provide data that will be added to the map
static_cities = ipyleaflet.GeoJSON(
    data=static_geojson,
    style={'color':'black','fillColor':'grey','fillOpacity':0.8, 'dashArray': '5,5'},
)

In [12]:
#Adding GeoJSON layer to the map
m.add_layer(static_cities)

In [13]:
#Adding layer control via dropdown widget to give the user the ability to toggle layers (i.e., crime types)
layer_drop = widgets.Dropdown(options=list(crime_df.columns), value= 'Total', description='Crime Type')
layer_drop.observe(choro_layer, 'value')

#Adding that as a widget control in ipyleaflet
layer_control = ipyleaflet.WidgetControl(widget=layer_drop, position='topright')

#Adding layer control to the map
m.add_control(layer_control)

In [14]:
#Creating an HTML widget that will be added to the map to display data
city_stat = widgets.HTML("Hover over a city to see its crime count for the selected crime type.")
html_control = ipyleaflet.WidgetControl(widget=city_stat, position='bottomright')
m.add_control(html_control)

In [15]:
#Adding the fullscreen control
m.add_control(ipyleaflet.FullScreenControl())

In [16]:
#Getting colors to add to legend control
colors = [linear.YlOrRd_09.rgb_hex_str(color) for color in linear.YlOrRd_09.index]
#Unpacking for legend control
low, medium, high = colors[0], colors[4], colors[8]

In [17]:
#Defining and adding legend control
legend = ipyleaflet.LegendControl({"Low":low, "Medium":medium, "High":high, "Data Not Provided": '#A9A9A9'}, name="Relative Crime Frequency", position="bottomleft")
m.add_control(legend)

In [18]:
#Creating callback function to display crime statistics for each city when hovered over
#Prints the name, crime types, and count for each city when hovered over or tells the user the city did not provide data
def update_city_stat(feature, **kwargs):
    if pd.isnull(crime_df.loc[feature['TOWN'], layer_drop.value]):
        val = "Data not provided by this city"
    else:
        val = str(int(crime_df.loc[feature['TOWN'], layer_drop.value]))
    city_stat.value = "City: " + str(feature['TOWN']) + "<br>" + "Instances of " + str(layer_drop.value) + ": " + val

In [19]:
#Telling ipyleaflet to watch for the user hovering over these layers and gives it a function to call when it happens
dyn_cities.on_hover(update_city_stat)

static_cities.on_hover(update_city_stat)

In [27]:
#Defining HTML widgets for labels
top_label = widgets.HTML(value = '''<h1><b>Data Visualization Challenge 2020</b></h1> <h3>Massachusetts 2018 NIBRS Data</h3>
 <ul>
  <li>Use the dropdown box in the right hand corner to select the crime type displayed</li>
  <li>Hover over a city to see the city's name and the number of recorded instances of the selected crime type</li>
  <li>Use the scroll wheel or the zoom control in the top left corner to zoom in and out of the map</li>
  <li>Use the fullscreen button in the top left corner below the zoom control to go fullscreen</li>
</ul> ''')

In [30]:
bottom_label = widgets.HTML(value = "<h3>Special thanks to Chad Brouillette for his assistance</h3> <h3>Credit to <a href=https://tereshenkov.wordpress.com/2017/11/08/removing-redundant-polyline-vertices-using-arcpy/><u style = 'color:blue;'>this</u></a> post for the script that uses collinearity to remove excessive coordinates from the map</h3>")

In [31]:
widgets.VBox([top_label, m, bottom_label])